In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister,ClassicalRegister
from qiskit_machine_learning.neural_networks import EstimatorQNN,SamplerQNN
from IPython.display import clear_output
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi
from qiskit.circuit import ParameterVector
from qiskit_algorithms.utils import algorithm_globals
import time 
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2,RealAmplitudes
import numpy as np

import pickle 
import time
from qiskit.circuit.library import iSwapGate
from math import comb

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
sqrt_iSWAP = iSwapGate().power(1/2)

from matplotlib import colormaps
list(colormaps)
cmap = 'gist_heat'

from Z2_2flavourClass import Z2_2flavour

In [103]:
# matter_defect = [7,12]
# nLinks = 7

matter_defect = [7,9]
nLinks = 5
nShots = 100000 

dtFactor = 0.2
Trotter_time = np.arange(dtFactor,10*dtFactor,dtFactor)
nTrotter_total = Trotter_time.shape[0]
dt_min = 0.4
nTrotter_list = np.ceil(Trotter_time/dt_min)
dt_list = Trotter_time/nTrotter_list

#Adding zero time
# Trotter_time = np.concatenate([np.array([0]),Trotter_time])
# nTrotter_list = np.concatenate([np.array([0]),nTrotter_list])
# dt_list = np.concatenate([np.array([dt_min]),dt_list])
print(nTrotter_list)


def getStringList(jobs_dict):
    string_list_dict = {}

    for key in jobs_dict.keys():
        print(key)
        l = jobs_dict[key]
        string_list = []
        for job_id in l:
            
            try:
                job = service.job(job_id)
                r = job.result()
                string = r[0].data.meas.get_counts()
                string_list.append(string)
                
            except Exception as e:
                print("Error in retriving:", job_id)

            
        string_list_dict[key] = string_list
    return string_list_dict

[1. 1. 2. 2. 3. 3. 4. 4. 5.]


In [80]:
nTrotter_list[i] + 1

6.0

In [81]:
depth = []
nCNOT = []
for i in range(nTrotter_list.shape[0]):
    z2 = Z2_2flavour(nLinks = nLinks,matter_defect = matter_defect)
    z2.getTrotterCircuit(1,nTrotter=int(nTrotter_list[i]),fFactor=0.2,dtFactor=dt_list[i],measure=True,rescale=False)
    circuit = z2.circuit
    depth.append(circuit.depth())
    nCNOT.append(circuit.count_ops()['xx_plus_yy']*2)
    print("nTrotter = ", nTrotter_list[i]+1,"depth = ",circuit.depth())
    print(circuit.count_ops()['xx_plus_yy']*2, "CNOT")

nTrotter =  2.0 depth =  40
120 CNOT
nTrotter =  2.0 depth =  40
120 CNOT
nTrotter =  3.0 depth =  77
240 CNOT
nTrotter =  3.0 depth =  77
240 CNOT
nTrotter =  4.0 depth =  114
360 CNOT
nTrotter =  4.0 depth =  114
360 CNOT
nTrotter =  5.0 depth =  151
480 CNOT
nTrotter =  5.0 depth =  151
480 CNOT
nTrotter =  6.0 depth =  188
600 CNOT


In [86]:
from qiskit_ibm_runtime import QiskitRuntimeService
# Save an IBM Quantum account and set it as your default account.
# service = QiskitRuntimeService(channel="ibm_quantum",instance="utokyo-kawasaki/internal/icepp-r5",token="")
# QiskitRuntimeService.save_account(channel="ibm_quantum",overwrite=True,instance="utokyo-kawasaki/internal/icepp-r5", token="86a8c6f00ee915610465242b9c54b2d98e97cf6e394a0de8014fa95a773b78becdd83b0cf0436a1d95d47ec83b0dc1cc008991f433b3d10166b3268cfffe953d", set_as_default=True)

service = QiskitRuntimeService(channel="ibm_quantum",instance="ibm-q-utokyo/internal/icepp-r5",token="86a8c6f00ee915610465242b9c54b2d98e97cf6e394a0de8014fa95a773b78becdd83b0cf0436a1d95d47ec83b0dc1cc008991f433b3d10166b3268cfffe953d")
QiskitRuntimeService.save_account(channel="ibm_quantum",overwrite=True,instance="ibm-q-utokyo/internal/icepp-r5", token="86a8c6f00ee915610465242b9c54b2d98e97cf6e394a0de8014fa95a773b78becdd83b0cf0436a1d95d47ec83b0dc1cc008991f433b3d10166b3268cfffe953d", set_as_default=True)

#service = QiskitRuntimeService()

service.backends()

# backend_name = "kawasaki"
backend_name = "torino"
backend = service.backend("ibm_"+backend_name)

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
backend.status()


In [87]:
n_jobs = nTrotter_list*10
print(sum(n_jobs))


250.0


In [88]:
#rescale jobs 
rescale_setting = True
jobs_rescale_dict = {}
with Session(backend=backend) as session:
    sampler_em = Sampler(mode = session)
    sampler_em.options.dynamical_decoupling.enable = True
    sampler_em.options.twirling.enable_gates = True


    for i in range(len(nTrotter_list)):
        nTrotter = int(nTrotter_list[i])
        if nTrotter%2==1:
            continue
        dt = dt_list[i]
        print("current trotter size = ", nTrotter," dt = ", dt, 't_total = ', dt*nTrotter," matter_defect = ",matter_defect)


        jobs_id_list = []
        for j in np.arange(0,n_jobs[i],1):

            z2 = Z2_2flavour(nLinks = nLinks,matter_defect = matter_defect)
            z2.getTrotterCircuit(1,nTrotter=nTrotter,fFactor=0.2,dtFactor=dt,measure=True,rescale=rescale_setting)
            circuit = z2.circuit
            

            isa_qc = pm.run(circuit)
            job = sampler_em.run([(isa_qc,[])],shots = nShots)

            print(f"job ID: {job.job_id()}")
            jobs_id_list.append(job.job_id())
            
        key = "nTrotter_"+str(nTrotter)+"_dtRescale_"+str(dt)
        jobs_rescale_dict[key] =  jobs_id_list


current trotter size =  2  dt =  0.30000000000000004 t_total =  0.6000000000000001  matter_defect =  [7, 9]
job ID: cz0pkfnteyw0008znxgg
job ID: cz0pkhpccdf0008har40
job ID: cz0pkjeccdf0008har4g
job ID: cz0pkk6q2bag008yc8k0
job ID: cz0pkmpteyw0008znxh0
job ID: cz0pknynr89g008chra0
job ID: cz0pkpprswz0008dnch0
job ID: cz0pkq6rswz0008dnchg
job ID: cz0pkqprswz0008dncj0
job ID: cz0pkrznr89g008chrag
job ID: cz0pkt7rswz0008dncjg
job ID: cz0pktqye310008800c0
job ID: cz0pkvqrswz0008dnckg
job ID: cz0pkwqnr89g008chrbg
job ID: cz0pkyfrswz0008dncm0
job ID: cz0pkz7q2bag008yc8kg
job ID: cz0pkzzye310008800e0
job ID: cz0pm0grswz0008dncmg
job ID: cz0pm18teyw0008znxhg
job ID: cz0pm28ccdf0008har7g
current trotter size =  2  dt =  0.4 t_total =  0.8  matter_defect =  [7, 9]
job ID: cz0pm2rrswz0008dncng
job ID: cz0pm3gccdf0008har80
job ID: cz0pm4gteyw0008znxj0
job ID: cz0pm5rccdf0008har8g
job ID: cz0pm6gteyw0008znxk0
job ID: cz0pm70ye310008800eg
job ID: cz0pm7rye310008800f0
job ID: cz0pm8hrswz0008dncp0
job

In [89]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')
rescale_name = backend_name+"_"+str(nLinks*3+2)+"Rescaling_"+datetime.today().strftime('%Y-%m-%d')+"_nCharges_"+str(len(matter_defect))
print(rescale_name+'.pkl')
# with open(rescale_name+'.pkl', 'wb') as file:
#     pickle.dump(jobs_rescale_dict, file)

torino_17Rescaling_2025-02-28_nCharges_2.pkl


In [96]:
rescale_name = 'torino_17Rescaling_2025-02-28_nCharges_2.pkl'

In [97]:
string_list_rescale = getStringList(jobs_rescale_dict)
print("stringList_"+rescale_name+'.pkl')

nTrotter_2_dtRescale_0.30000000000000004
cz0pkfnteyw0008znxgg
cz0pkhpccdf0008har40
cz0pkjeccdf0008har4g
cz0pkk6q2bag008yc8k0
cz0pkmpteyw0008znxh0
cz0pknynr89g008chra0
cz0pkpprswz0008dnch0
cz0pkq6rswz0008dnchg
cz0pkqprswz0008dncj0
cz0pkrznr89g008chrag
cz0pkt7rswz0008dncjg
cz0pktqye310008800c0
cz0pkvqrswz0008dnckg
cz0pkwqnr89g008chrbg
cz0pkyfrswz0008dncm0
cz0pkz7q2bag008yc8kg
cz0pkzzye310008800e0
cz0pm0grswz0008dncmg
cz0pm18teyw0008znxhg
cz0pm28ccdf0008har7g
nTrotter_2_dtRescale_0.4
cz0pm2rrswz0008dncng
cz0pm3gccdf0008har80
cz0pm4gteyw0008znxj0
cz0pm5rccdf0008har8g
cz0pm6gteyw0008znxk0
cz0pm70ye310008800eg
cz0pm7rye310008800f0
cz0pm8hrswz0008dncp0
cz0pm91ye310008800fg
cz0pm9sye310008800g0
cz0pma9rswz0008dncpg
cz0pmasteyw0008znxmg
cz0pmbhq2bag008yc8m0
cz0pmc9ye310008800gg
cz0pmcsrswz0008dncq0
cz0pmdsnr89g008chre0
cz0pmesrswz0008dncqg
cz0pmfhrswz0008dncr0
cz0pmgateyw0008znxn0
cz0pmgtye310008800h0
nTrotter_4_dtRescale_0.35000000000000003
cz0pmhjye310008800hg
cz0pmjaccdf0008har9g
cz0pmkjnr89

In [98]:
print("stringList_"+rescale_name+'.pkl')
with open("stringList_"+rescale_name+'.pkl', 'wb') as file:
    pickle.dump(string_list_rescale, file)

stringList_torino_17Rescaling_2025-02-28_nCharges_2.pkl.pkl


In [94]:
# #rescale jobs 
# rescale_setting = False
# jobs_evo_dict = {}
# with Session(backend=backend) as session:
#     sampler_em = Sampler(mode = session)
#     sampler_em.options.dynamical_decoupling.enable = True
#     sampler_em.options.twirling.enable_gates = True


#     for i in range(len(nTrotter_list)):
#         nTrotter = int(nTrotter_list[i])

#         dt = dt_list[i]
#         print("current trotter size = ", nTrotter," dt = ", dt, 't_total = ', dt*nTrotter," matter_defect = ",matter_defect)


#         jobs_id_list = []
#         for j in np.arange(0,n_jobs[i],1):

#             z2 = Z2_2flavour(nLinks = nLinks,matter_defect = matter_defect)
#             z2.getTrotterCircuit(1,nTrotter=nTrotter,fFactor=0.2,dtFactor=dt,measure=True,rescale=rescale_setting)
#             circuit = z2.circuit
            

#             isa_qc = pm.run(circuit)
#             job = sampler_em.run([(isa_qc,[])],shots = nShots)

#             print(f"job ID: {job.job_id()}")
#             jobs_id_list.append(job.job_id())
            
#         key = "nTrotter_"+str(nTrotter)+"_dtRescale_"+str(dt)
#         jobs_evo_dict[key] =  jobs_id_list


current trotter size =  1  dt =  0.2 t_total =  0.2  matter_defect =  [7, 9]
job ID: cz0pq04ccdf0008harkg
job ID: cz0pq0wnr89g008chrt0
job ID: cz0pq1wye310008800xg
job ID: cz0pq2mccdf0008harm0
job ID: cz0pq34q2bag008yc950
job ID: cz0pq3wnr89g008chrtg
job ID: cz0pq4wnr89g008chrv0
job ID: cz0pq5mq2bag008yc95g
job ID: cz0pq6cccdf0008harmg
job ID: cz0pq6wrswz0008dnda0
current trotter size =  1  dt =  0.4 t_total =  0.4  matter_defect =  [7, 9]
job ID: cz0pq7mnr89g008chrvg
job ID: cz0pq85rswz0008dndag
job ID: cz0pq8nq2bag008yc960
job ID: cz0pq95q2bag008yc96g
job ID: cz0pq9xccdf0008harng
job ID: cz0pqannr89g008chrw0
job ID: cz0pqbdrswz0008dndb0
job ID: cz0pqcdteyw0008zny0g
job ID: cz0pqddteyw0008zny1g
job ID: cz0pqedrswz0008dndbg
current trotter size =  2  dt =  0.30000000000000004 t_total =  0.6000000000000001  matter_defect =  [7, 9]
job ID: cz0pqexrswz0008dndc0
job ID: cz0pqfdq2bag008yc970
job ID: cz0pqgenr89g008chrwg
job ID: cz0pqhpccdf0008harp0
job ID: cz0pqjeye310008800yg
job ID: cz0pq

In [95]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')
evo_name = backend_name+"_"+str(nLinks*3+2)+"Evo_"+datetime.today().strftime('%Y-%m-%d')+"_nCharges_"+str(len(matter_defect))
print(evo_name+'.pkl')


torino_17Evo_2025-02-28_nCharges_2.pkl


In [99]:
evo_name = 'torino_17Evo_2025-02-28_nCharges_2.pkl'

In [100]:
with open(evo_name+'.pkl', 'wb') as file:
    pickle.dump(jobs_evo_dict, file)

In [104]:
string_list_evo = getStringList(jobs_evo_dict)
print("stringList_"+evo_name+'.pkl')

nTrotter_1_dtRescale_0.2
nTrotter_1_dtRescale_0.4
nTrotter_2_dtRescale_0.30000000000000004
nTrotter_2_dtRescale_0.4
nTrotter_3_dtRescale_0.3333333333333333
nTrotter_3_dtRescale_0.39999999999999997
nTrotter_4_dtRescale_0.35000000000000003
nTrotter_4_dtRescale_0.4
nTrotter_5_dtRescale_0.36
Error in retriving: cz0pwkjye310008801qg
stringList_torino_17Evo_2025-02-28_nCharges_2.pkl.pkl


In [105]:

with open("stringList_"+evo_name+'.pkl', 'wb') as file:
    pickle.dump(string_list_evo, file)